In [5]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv2D,MaxPooling2D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
import pylab as plt
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
import time
from keras.utils import np_utils
from sklearn import preprocessing as pp

import h5py

X_train = np.load("./data_dump/train_features.pkl")
Y_train = np.load("./data_dump/train_labels.pkl")
X_test = np.load("./data_dump/test_features.pkl")
Y_test = np.load("./data_dump/test_labels.pkl")



cnn2d_shallow = Sequential([Conv2D(64, (3,3), padding="same",input_shape=(8,8,1)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(1),
    Activation('sigmoid'),
])

cnn2d_moderate = Sequential([Conv2D(64, (3,3), padding="same",input_shape=(8,8,1)),
    Activation('relu'),
    Conv2D(128, (3,3), padding="same"),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dropout(0.5),
    Dense(1),
    Activation('sigmoid'),
])

cnn2d_deep = Sequential([Conv2D(64, (3,3), padding="same",input_shape=(8,8,1)),
    Activation('relu'),
    Conv2D(128, (3,3), padding="same"),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(256, (3,3), padding="same"),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dropout(0.5),
    Dense(1),
    Activation('sigmoid'),
])
print(cnn2d_moderate.summary())
print(cnn2d_deep.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 8, 8, 64)          640       
_________________________________________________________________
activation_34 (Activation)   (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 8, 8, 128)         73856     
_________________________________________________________________
activation_35 (Activation)   (None, 8, 8, 128)         0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 128)               262272    
__________

In [6]:
cnn2d_shallow.compile(loss='binary_crossentropy', optimizer="adam",metrics=['accuracy'])
cnn2d_moderate.compile(loss='binary_crossentropy', optimizer="adam",metrics=['accuracy'])
cnn2d_deep.compile(loss='binary_crossentropy', optimizer="adam",metrics=['accuracy'])

# train
cnn2d_shallow.fit(X_train, Y_train, batch_size=64,epochs=5)
print("------------------------------------------------------------------------------------------------------")
cnn2d_moderate.fit(X_train, Y_train, batch_size=64,epochs=5)
print("------------------------------------------------------------------------------------------------------")
cnn2d_deep.fit(X_train, Y_train, batch_size=64,epochs=5)
print("------------------------------------------------------------------------------------------------------")

# serialize weights to HDF5
cnn2d_shallow.save("./models/NSL_KDD_Train_shallow.h5")
cnn2d_moderate.save("./models/NSL_KDD_Train_moderate.h5")
cnn2d_deep.save("./models/NSL_KDD_Train_deep.h5")

Epoch 1/5
125972/125972 [==============================] - 11s 83us/step - loss: 0.6911 - acc: 0.5342
Epoch 2/5
125972/125972 [==============================] - 10s 77us/step - loss: 0.6908 - acc: 0.5346
Epoch 3/5
125972/125972 [==============================] - 10s 82us/step - loss: 0.6908 - acc: 0.5346
Epoch 4/5
125972/125972 [==============================] - 10s 79us/step - loss: 0.6908 - acc: 0.5346
Epoch 5/5
125972/125972 [==============================] - 9s 75us/step - loss: 0.6908 - acc: 0.5346
------------------------------------------------------------------------------------------------------
Epoch 1/5
125972/125972 [==============================] - 60s 474us/step - loss: 0.6910 - acc: 0.5344
Epoch 2/5
125972/125972 [==============================] - 59s 468us/step - loss: 0.6908 - acc: 0.5346
Epoch 3/5
125972/125972 [==============================] - 60s 477us/step - loss: 0.6908 - acc: 0.5346
Epoch 4/5
125972/125972 [==============================] - 60s 476us/step - los

### Evaluation Metrics - F1, Confusion Matrix - (Normal as 0 and Anomaly as 1)

In [8]:
from sklearn.metrics import f1_score
import pandas as pd 
from sklearn.metrics import classification_report


shallow_pred = cnn2d_shallow.predict_classes(X_test)
moderate_pred = cnn2d_moderate.predict_classes(X_test)
deep_pred = cnn2d_deep.predict_classes(X_test)

print(classification_report(Y_test, shallow_pred))
print(classification_report(Y_test, moderate_pred))
print(classification_report(Y_test, deep_pred))

print(f1_score(Y_test, shallow_pred))
print(f1_score(Y_test, moderate_pred))
print(f1_score(Y_test, deep_pred))

             precision    recall  f1-score   support

          0       0.43      1.00      0.60      9711
          1       0.00      0.00      0.00     12832

avg / total       0.19      0.43      0.26     22543

             precision    recall  f1-score   support

          0       0.43      1.00      0.60      9711
          1       0.00      0.00      0.00     12832

avg / total       0.19      0.43      0.26     22543

             precision    recall  f1-score   support

          0       0.43      1.00      0.60      9711
          1       0.00      0.00      0.00     12832

avg / total       0.19      0.43      0.26     22543

0.0
0.0
0.0


C:\Anaconda\envs\tensorflow-cpu\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\envs\tensorflow-cpu\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
